In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Define the custom separators
separator = '\x09'
set_separator = ','

# Specify the path to the dataset file
dataset_file = "data/opt/Malware-Project/BigDataset/IoTScenarios/CTU-IoT-Malware-Capture-21-1/bro/conn.log.labeled"

df = pd.read_csv(dataset_file, sep='\x09', header=None, comment='#')




In [3]:
last_three_columns = df[20]
benign_value = last_three_columns[0]

# make this series into three different columns. The first element is a column, the second element is a column, and the third element is a column
last_three_columns = last_three_columns.str.split("   ", expand=True)
last_three_columns

# now, integrate the three columns into the original dataframe
df[20] = last_three_columns[0]
df[21] = last_three_columns[1]
df[22] = last_three_columns[2]

In [4]:
# rename the columns
with open(dataset_file, 'r') as f:
    # initiate the for loop
    for line in f:
        if line.startswith('#fields'):
            fields = line.strip()[8:].split(separator)
            fields = [field.strip() for field in fields]
            print(fields)
            df.columns = fields
        elif line.startswith('#'):
            continue
        else:
            break



['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents', 'label', 'detailed-label']


In [5]:
# map the target labels to 0
df['label'] = df['label'].map({"Benign": 0, "Malicious": 1})

# drop nan values
df.dropna(inplace=True)

In [31]:
# create a new dataframe with only the useful columns
df_reduced = df[['id.orig_h', 'id.resp_h', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents', 'label', 'detailed-label']]

In [32]:
# convert the duration column to float
df_reduced["duration"] = df_reduced["duration"].map(lambda x: float(x) if x != "-" else 0.0)

/tmp/ipykernel_22654/713016053.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced["duration"] = df_reduced["duration"].map(lambda x: float(x) if x != "-" else 0.0)


In [33]:
# remove id.orig_h, local_orig and local_resp because it is the same for all entries
df_reduced.drop(['id.orig_h', "local_orig", "local_resp", "tunnel_parents", "detailed-label"], axis=1, inplace=True)



/tmp/ipykernel_22654/695082480.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced.drop(['id.orig_h', "local_orig", "local_resp", "tunnel_parents", "detailed-label"], axis=1, inplace=True)


In [36]:
from sklearn.preprocessing import LabelEncoder
# convert proto and service to categorical columns

encoder_proto = LabelEncoder()
df_reduced["proto"] = encoder_proto.fit_transform(df_reduced["proto"])

encoder_service = LabelEncoder()
df_reduced["service"] = encoder_service.fit_transform(df_reduced["service"])

encoder_conn_state = LabelEncoder()
df_reduced["conn_state"] = encoder_conn_state.fit_transform(df_reduced["conn_state"])

/tmp/ipykernel_22654/4207215523.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced["proto"] = encoder_proto.fit_transform(df_reduced["proto"])
/tmp/ipykernel_22654/4207215523.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced["service"] = encoder_service.fit_transform(df_reduced["service"])
/tmp/ipykernel_22654/4207215523.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [37]:
df_reduced

,id.resp_h,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,192.168.100.1,2,1,0.000000,-,-,2,0,D,1,67,0,0,0
1,192.168.100.1,2,1,0.001494,90,90,5,0,Dd,2,146,2,146,0
2,192.168.100.1,2,1,5.005135,78,0,2,0,D,2,134,0,0,0
3,192.168.100.1,2,1,0.001483,90,90,5,0,Dd,2,146,2,146,0
4,192.168.100.1,2,1,5.038625,117,142,5,0,Dd,3,201,2,198,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3281,213.199.225.40,2,0,0.000000,-,-,2,0,D,1,76,0,0,0
3282,89.221.212.46,2,0,0.000000,-,-,2,0,D,1,76,0,0,0
3283,77.78.107.252,2,0,0.001990,48,48,5,0,Dd,1,76,1,76,0
3284,81.2.254.32,2,0,0.000000,-,-,2,0,D,1,76,0,0,0
